In [ ]:
!nvidia-smi

In [ ]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'tensorflow-text==2.15.0' np_utils swifter

In [ ]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob


TRACE = False
embedding_dim = 300
n_channels = 64
p_dropout = 0.2
epochs=1000
batch_size = 500
corpus_size=100000
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

In [ ]:
%%writefile get_data.sh
if [ ! -f news.csv ]; then
  wget -O news.csv https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
fi


In [ ]:
!bash get_data.sh

In [ ]:
path = './news.csv'
news_pre = pd.read_csv(path, header=0).sample(n=corpus_size).reset_index(drop=True)

In [ ]:
# We will reuse what we learned in previous labs!
def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [ ]:
import swifter
news = news_pre.title.swifter.apply(preprocess_text)
news.to_csv('news_processed.csv', header=False, index=False)

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'news_processed.csv'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield preprocess_text(line, should_join=False)

import gensim.models

sentences = MyCorpus()
word2vec = gensim.models.Word2Vec(sentences=sentences, vector_size=embedding_dim, epochs=10)
word2vec_model = word2vec.wv

In [ ]:
weights = word2vec_model.vectors
weights = tf.cast(tf.constant(weights), tf.float32)
vocab_size = len(word2vec_model.index_to_key)

In [ ]:
weights.shape



In [ ]:
news_preprocessed = pd.DataFrame()
news_preprocessed['label'] = news_pre.category.map({'Business': 0, 'Sports': 1, 'Sci/Tech': 2, 'World': 3})
news_preprocessed['title'] = news
news_preprocessed

In [ ]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(textblob_tokenizer(row.title))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(news_preprocessed)

In [ ]:
X = np.zeros((len(news_preprocessed), maximum))
for index, row in news_preprocessed.iterrows():
  word_ix = 0
  for word in textblob_tokenizer(row.title):
    token = vocab_size
    if word in word2vec_model.key_to_index:
      token = word2vec_model.key_to_index[word]
    X[index, word_ix] = token
    word_ix += 1

y = news_preprocessed.label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = tf.cast(tf.constant(X_train), tf.float32)
X_test = tf.cast(tf.constant(X_test), tf.float32)
y_train = tf.one_hot(tf.constant(y_train), 4)
y_test = tf.one_hot(tf.constant(y_test), 4)

In [ ]:
X_train[0]

In [ ]:
weights.shape

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=weights.shape[0], output_dim=embedding_dim, input_length=maximum, embeddings_initializer=Constant(weights), trainable=True))
model.add(Conv1D(filters=n_channels,kernel_size = 3, padding='same',data_format="channels_first"))  # Add a Conv1d layer tp have n_channels filters of 3x3. It is key you ensure you don't loose dimension size!
model.add(MaxPooling1D(pool_size=2, strides=None, padding="valid", data_format="channels_last"))
model.add(ELU(alpha=1.0))  # Add an activation layer
model.add(Conv1D(filters=2*n_channels, kernel_size=3, padding='same', data_format='channels_first')) # Add another Conv1d layer tp have 2*n_channels filters of 3x3. It is key you ensure you don't loose dimension size!
model.add(MaxPooling1D(pool_size=2, strides=None, padding="valid", data_format="channels_last"))
model.add(ELU(alpha=1.0))  # Add an activation layer
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(None, embedding_dim,)))  # Average out the convolution dimension
model.add(Dense(100, activation='relu'))  # Add a couple of Dense Layers with RELU or leaky_relu activations. You may add Batch Norm if you want too
model.add(Dense(50, activation='tanh'))
Dropout(.2, input_shape=(embedding_dim,))  # Add a dropout
model.add(Dense(4, activation='softmax'))   # Final Dense layer

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), optimizer='adam', metrics='accuracy')
model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, min_delta=0.01)
history = model.fit(x=X_train, y=y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), workers=5, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=2.0)

In [ ]:
plot_metrics(history.history['accuracy'], history.history['val_accuracy'], "Accuracy", "Acurracy", ylim=1.0)

In [ ]:
x_val = np.zeros((2, maximum))
for index, row in enumerate(['supercomputer will put workers jobless soon', 'patriots goes winning super bowl']):
    ix = 0
    for word in tokenizer(row):
        if word not in word2vec_model:
            representation = 0
        else:
            representation = word2vec_model.index_to_key.index(word)
        x_val[index, ix] = representation
        ix += 1
y_val = tf.one_hot([0,1], depth=4)

In [ ]:
x_val

In [ ]:
y_val

In [ ]:
model.predict(x_val)

In [ ]:
tf.math.argmax(model.predict(x_val), axis=1)

In [ ]:
tf.math.argmax(model.predict(x_val), axis=1) == tf.math.argmax(y_val, axis=1)

In [ ]:
model.evaluate(X_test, y_test)